In [1]:
from src.data.get_health_units import GetHealthUnits
from src.visualization.map.health_units_map import HealthUnitsMap

health_units = GetHealthUnits.get()

#hu_map = HealthUnitsMap(health_units)
#hu_map.generateMap()

In [ ]:
from src.visualization.heatmap.health_units_heatmap import HealthUnitsHeatmap

hu_heatmap = HealthUnitsHeatmap(health_units)
hu_heatmap.generateMap()

In [ ]:
from src.data.get_mun_schools import GetMunicipalSchools
from src.visualization.map.mun_schools_map import MunicipalSchoolsMap

schools = GetMunicipalSchools.get()

ms_map = MunicipalSchoolsMap(schools)
ms_map.generateMap()

In [ ]:
from src.visualization.heatmap.mun_schools_heatmap import MunicipalSchoolsHeatmap

ms_heatmap = MunicipalSchoolsHeatmap(schools)
ms_heatmap.generateMap()

In [98]:
from src.visualization.charts.practitioners_by_district import PractitionersByDistrictPlot

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

from bokeh.layouts import widgetbox, row, column
from bokeh.models import Select
from bokeh.charts import Bar
from bokeh.charts.attributes import cat
from bokeh.charts.operations import blend
from bokeh.models import ColumnDataSource

from bokeh.palettes import PuBu
from bokeh.models import ColumnDataSource, ranges, LabelSet
from bokeh.plotting import figure

import pandas as pd

def getEmployees():
    d = {}
    for u in health_units:
        if u.district.title() in d:
            d[u.district.title()] += u.employees
        else:
            d[u.district.title()] = u.employees
    return d

def getUnits():
    d = {}
    for u in health_units:
        if u.district.title() in d:
            d[u.district.title()] += 1
        else:
            d[u.district.title()] = 1
    return d

def modify_doc(doc):

    table = getEmployees()
    table = {
        'district' : [x for x in sorted(table.keys())],
        'employees' : [table[x] for x in sorted(table.keys())]
    }
    source = ColumnDataSource(table)
    
    
    # Define the callback function: update_plot
    def update_plot(attr, old, new):
        if new == 'Employees x District':
            table = getEmployees()
            source.data = {
                'district' : [x for x in sorted(table.keys())],
                'employees' : [table[x] for x in sorted(table.keys())]
            }
            plot.vbar(source=source,x='district',top='employees',bottom=0,width=0.3,color=PuBu[7][2])
            #plot.x_range = source.data['district']
            #plot.y_range = ranges.Range1d(start=0, end = max(source.data['employees']))
            print(max(source.data['employees']))
        else:
            table = getUnits()
            source.data = {
                'district' : [x for x in sorted(table.keys())],
                'employees' : [table[x] for x in sorted(table.keys())]
            }
            plot.vbar(source=source,x='district',top='employees',bottom=0,width=0.3,color=PuBu[7][2])
            #plot.x_range = source.data['district']
            print(max(source.data['employees']))
            #plot.y_range = ranges.Range1d(start=0, end = max(source.data['employees']))

    # Create a dropdown Select widget for the x data: x_select
    plot_select = Select (
        options = ['Employees x District', 'Units x District'],
        value = 'Employees x District',
        title = 'Plot'
    )

    # Attach the callback to the 'value' property of selects
    plot_select.on_change('value', update_plot)
    
    plot = figure(plot_width=600, plot_height=400, tools="save",
            x_axis_label = "",
            y_axis_label = "Employees",
            title = "Employees by District",
#            x_minor_ticks = 5,
            x_range = source.data['district'])
#            y_range= ranges.Range1d(start=0, end = max(source.data['employees'])))

    plot.vbar(source=source,x='district',top='employees',bottom=0,width=0.3,color=PuBu[7][2])
    #plot.add_layout(labels)
    
    # Make a row layout of widgetbox(slider) and p and add it to the current document
    layout = column(widgetbox(plot_select), plot)
    
    doc.add_root(layout)



handler = FunctionHandler(modify_doc)
app = Application(handler)

from tornado.ioloop import IOLoop
loop = IOLoop.current()

In [99]:
def show_app(app, notebook_url="127.0.0.1:8888"):
    from IPython.display import HTML, display
    from bokeh.embed import autoload_server
    from bokeh.server.server import Server
    
    server = Server({'/': app}, io_loop=loop, port=0, host='*', allow_websocket_origin=[notebook_url])
    server.start()
    
    script = autoload_server(model=None, url='http://127.0.0.1:%d' % server.port)
    
    display(HTML(script))

In [100]:
show_app(app)

INFO:bokeh.server.server:Starting Bokeh server version 0.12.4


INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=7125cc6e-9855-4fba-a645-24bb6fd6a82e&_=1492274047527 (127.0.0.1) 45.16ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=9d3af13f493d36073a89714f6a5240c6 (127.0.0.1) 5.63ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=1af1302b8bd7fcc88c7bcafb8771497b (127.0.0.1) 2.91ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
INFO:tornado.access:200 GET /static/css/bokeh.min.css?v=7246afcfffc127faef7c138bce4742e9 (127.0.0.1) 2.17ms
INFO:tornado.access:200 GET /static/css/bokeh-widgets.min.css?v=d9cb9322d940f107727b091ff98d9c70 (127.0.0.1) 2.39ms


11
383


In [ ]:
from bokeh.layouts import widgetbox, row
from bokeh.models import Select
from bokeh.charts import Bar
from bokeh.charts.attributes import cat
from bokeh.charts.operations import blend
from bokeh.io import output_notebook, show

def getPractitioners(district):
    d = {}
    for u in health_units:
        if u.district == district:
            for field in u.practitioners:
                if field in d:
                    d[field] += u.practitioners[field]
                else:
                    d[field] = u.practitioners[field]
    return d

d = getPractitioners('Norte I')

table = {
    'expertise' : [x for x in sorted(d.keys())],
    'amount' : [d[x] for x in sorted(d.keys())]
}

# Attach the callback to the 'value' property of selects
#district_select.on_change('value', update_plot)        

# Add the color mapper to the circle glyph
bar = Bar(\
        table,
        values = blend('amount', name = 'amount', labels_name = 'amount'),\
        label = cat(columns = 'expertise'),\
        stack = cat(columns = 'amount'),\
        agg = 'mean',\
        title = 'Teste ',\
        legend = False)#),\
        #tooltips = [('Average', '@average{1.11}' + ' g/km')])

# Make a row layout of widgetbox(slider) and p and add it to the current document
#layout = row(widgetbox(district_select), bar)

output_notebook()
show(bar)

In [14]:
from bokeh.io import output_notebook, show
from bokeh.charts import Donut
from bokeh.charts.utils import df_from_json
from bokeh.sampledata.olympics2014 import data

import pandas as pd

# utilize utility to make it easy to get json/dict data converted to a dataframe
df = df_from_json(data)

# filter by countries with at least one medal and sort by total medals
df = df[df['total'] > 8]
df = df.sort_values(by="total", ascending=False)

df = pd.melt(df, id_vars=['abbr'],
             value_vars=['bronze', 'silver', 'gold'],
             value_name='medal_count', var_name='medal')
#print(df.groupby(['medal']))
print(df)
# original example
#d = Donut(df, label=['abbr', 'medal'], values='medal_count',
#          text_font_size='8pt', hover_text='medal_count')
d = Donut(df.groupby(['medal']), label=['medal'], values='medal_count',
          text_font_size='8pt', hover_text='medal_count')

output_notebook()
show(d)

   abbr   medal  medal_count
0   RUS  bronze            7
1   NLD  bronze            8
2   USA  bronze           10
3   NOR  bronze            7
4   CAN  bronze            4
5   DEU  bronze            4
6   FRA  bronze            5
7   CHE  bronze            1
8   AUT  bronze            1
9   SWE  bronze            2
10  RUS  silver            8
11  NLD  silver            6
12  USA  silver            4
13  NOR  silver            4
14  CAN  silver            8
15  DEU  silver            3
16  FRA  silver            2
17  CHE  silver            3
18  AUT  silver            6
19  SWE  silver            5
20  RUS    gold            6
21  NLD    gold            6
22  USA    gold            6
23  NOR    gold            8
24  CAN    gold            4
25  DEU    gold            8
26  FRA    gold            3
27  CHE    gold            6
28  AUT    gold            2
29  SWE    gold            2


TypeError: 'NoneType' object is not iterable